# Homework 3

In this homework we will practice basic analysis such as feature extraction and data transforms (Fourier, wavelet), and feature extraction.


**1. Ice-shelf seismograms** (10 points)

Time-domain filtering, 1D Fourier transform.

**2. TEC from the Hunga-Tunga explosion** (10 points)

Time-domain filtering, 1D wavelet transform.

**3. 2D Crustal model** (10 points)

practice reading netcdf, making maps and exploring 2D spectral content.



## 1) Time Frequency analysis of iceshelf vibrations 

We will explore the spectral content of the vibrations felt on iceshelves. We first download seismic data, then filter it at different frequency bandwidths, then plot the spectrogram and comment on the data.

The seismic data is handled by the Obspy package. Review the obspy tutorial that Ariane.
We will download the data presented in: Aster, R.C., Lipovsky, B.P., Cole, H.M., Bromirski, P.D., Gerstoft, P., Nyblade, A., Wiens, D.A. and Stephen, R., 2021. Swell‐Triggered Seismicity at the Near‐Front Damage Zone of the Ross Ice Shelf. Seismological Research Letters. https://doi.org/10.1785/0220200478

__Tips__:
1. Check out the SciPy filtering help here: https://scipy-cookbook.readthedocs.io/items/ButterworthBandpass.html. Obspy has built in functions as well, but for the sake of practicing, explore the scipy filtering functions.

2. The usual steps to handling seismic data are: data download (``get_waveforms``) & removing the instrumental response (``remove_response``).




**a. Import the relevant Obspy python modules (1 point).**

In [42]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import netCDF4 as nc
import numpy as np
import pandas as pd
import rasterio
import wget
import requests, zipfile , os, io

from rasterio.mask import mask
from rasterio.plot import show

 



**b. Data download (1 point)**

We will now download the data from station "DR01" from seismic network "XH", channel "LHN" from 1/1/2015 until 3/31/2015. The client will be the "IRIS" data center. Obspy functions take on UTCDateTime formatted obspy datetime object, be sure to call or import that specific function. (1 point)

In [43]:
# Import the Obspy modules to download seismic data
import obspy
import obspy.clients.fdsn.client as fdsn
from obspy import UTCDateTime

In [59]:
# answer below
# Download seismic data
network = 'XH'
station = 'DR01'
channel = 'LHN'# this channel gives a low frequency, 1Hz signal.
loc = ''
Tstart = UTCDateTime(year=2015, month=1, day=1)
Tend = UTCDateTime(year=2015, month=3, day=31)
fdsn_client = fdsn.Client('IRIS') # client to query the IRIS DMC server



In [60]:
Z = fdsn_client.get_waveforms(network=network, client=fdsn_client, station=station, location='--', channel=channel, starttime=Tstart,
                              endtime=Tend, attach_response=True, loc=loc)
# Merging data for  gaps, detrend, taper and remiving the seismic instrumental responce  to get to ground motion velocity units

Z.merge()
Z.detrend(type='linear')
Z[0].taper(max_percentage=0.05)
Z[0].remove_response()

# call to download noise waveforms
N = fdsn_client.get_waveforms(network=network, station=station, location='--', channel=channel, starttime=Tstart-7200,
    endtime=Tstart, attach_response=True)
# Merging data for  gaps, detrend, taper and remiving the seismic instrumental responce  to get to ground motion velocity units

N.merge()
N.detrend(type='linear')
N[0].taper(max_percentage=0.05)
N[0].remove_response()

ValueError: The current client does not have a dataselect service.

**c. Time series filtering (2 points)**

Now we will filter the trace to explore its frequency content. We will apply 3 filters:
1. a ``lowpass`` filter to look at seismic frequencies below 0.01Hz, or 100 s period

2. a ``bandpass`` filter to look at seismic frequencies between 0.01Hz-0.1 Hz (10-100s)

3. a ``highpass`` filter to look at seismic frequencies higher than 0.1 Hz (10s) and until the time series Nyquist frequency (0.5Hz since the data is sampled at 1 Hz).

In [61]:
import scipy.signal as signal

In [62]:
# answer below
from scipy.signal import butter,buttord,  sosfiltfilt, freqs

#sampling rate of the data:
fs = Z[0].stats.sampling_rate
#signal
z=np.asarray(Z[0].data)
#noise
n=np.asarray(N[0].data)

NameError: name 'Z' is not defined

In [63]:
#Butterworth filter to select the spectral content of the waveform. 


####First lowpass #####

#second order section = sos
# 2 is the order of the slop the [] is the range of hertz (0 wouldn't work so choose small value near)
sos_lp = signal.butter(2,[0.01], 'lp', fs=fs, output='sos')

zf_lp = signal.sosfiltfilt(sos_lp, z)
nf_lp = signal.sosfiltfilt(sos_lp, n)
t_lp=np.arange(0.,7200.,1./fs)

NameError: name 'fs' is not defined

In [64]:
#### Band Pass #####

#second order section = sos
# 2 is the order of the slop the [] is the range of hertz (0 wouldn't work so choose small value near)
sos_bp = signal.butter(2,(0.01,0.1), btype='bp', fs=fs, output='sos')

zf_bp = signal.sosfiltfilt(sos_bp, z)
nf_bp = signal.sosfiltfilt(sos_bp, n)
t_bp=np.arange(0.,7200.,1./fs)

NameError: name 'fs' is not defined

In [ ]:
#### high Pass #####

#second order section = sos
# 2 is the order of the slop the [] is the range of hertz (0 wouldn't work so choose small value near)
sos_hp = signal.butter(2,[0.1], 'hp', fs=fs, output='sos')

zf_hp = signal.sosfiltfilt(sos_hp, z)
nf_hp = signal.sosfiltfilt(sos_hp, n)
t_hp=np.arange(0.,7200.,1./fs)

**c. Fourier transform (3 points)**


Perform and the Fourier amplitude spectrum of the seismogram. Don't forget to label the figure properly! Use the Fourier frequency vector for x-axis. Use the tutorials for inspirtion.

In [ ]:
# import FFT modules
from scipy.fftpack import fft, ifft, fftfreq, next_fast_len
# answer below


npts = Z[0].stats.npts
## FFT the signals
# fill up until 2^N value to speed up the FFT
Nfft = next_fast_len(int(Z[0].data.shape[0])) # this will be an even number
freqVec = fftfreq(Nfft, d=Z[0].stats.delta)[:Nfft//2]
Z.taper(max_percentage=0.05)
Zhat = fft(Z[0].data,n=Nfft)#/np.sqrt(Z[0].stats.npts)

In [ ]:
fig,ax=plt.subplots(2,1,figsize=(10,8))

ax[0].plot(freqVec, np.abs(Zhat[:Nfft//2])/Nfft)
ax[0].grid(True)
ax[0].set_xscale('log');ax[0].set_yscale('log')
ax[0].set_xlabel('Frequency (Hz)');ax[0].set_ylabel('Amplitude (m/s)')

ax[1].hist(np.angle(Zhat))
ax[1].grid(True)

Comment on the spectral content of the seismograms. How does the relative contribution of the low, intermediate, and high frequency signal compares with the relative amplitude observed in the bandpass filtered time series?

**All of the frequencies are below 1 HZ, and exist mostly between 10^-5 HZ and 0.05 HZ. So a highpass has some of the data, the bandpass has some more than the highpass and the low pass has the most activity.**


**d. Synthetic noise (3 points)**

We have now a good idea of what the amplitude of seismic waves are at this station. Now create a noise signal using the Fourier amplitude spectrum of the seismic signal, and with a random phase. You can use the notes from our first Numpy example (2.7_data_transforms.ipynb)

In [ ]:
z=np.asarray(Z[0].data)

In [ ]:
from numpy import random
import numpy.random as random
from scipy.fftpack import ifft,ifftshift 

t=np.arange(0,len(z)-1,1./fs)
new_noise= random.uniform(-1,1,len(t))



In [ ]:
SNR=100.
plt.plot(t,z[:-1]/np.max(np.abs(z[:-1])) + new_noise/SNR);

In [ ]:
SNR=10
plt.plot(t,z[:-1]/np.max(np.abs(z[:-1])) + new_noise/SNR);

In [ ]:
SNR=5
plt.plot(t,z[:-1]/np.max(np.abs(z[:-1])) + new_noise/SNR);

In [ ]:
Nhat = fft(N[0].data,n=Nfft)


In [ ]:
#Final noise!
from scipy.fftpack import ifft

Nhat = fft(N[0].data,n=Nfft)

new_stuff=np.zeros(Nfft)

for i in range(Nfft//2):
    new_stuff[i] = np.abs(Nhat[i])*random.uniform(-np.pi,np.pi)

stuff = ifft(new_stuff)
plt.plot(stuff)

**e. !Sanity check! (1 point)**

Check that the Fourier amplitude spectrum of the noise is that of the original window. Overlay them on a plot 

In [ ]:
# answer below
npts = Z[0].stats.npts-1
## FFT the signals
# fill up until 2^N value to speed up the FFT
Nfft = next_fast_len(int(Z[0].data.shape[0]-1)) # this will be an even number
freqVec = fftfreq(Nfft, d=Z[0].stats.delta)[:Nfft//2]

Nat = fft(new_noise,n=Nfft)#/np.sqrt(Z[0].stats.npts)

plt.plot(freqVec,np.abs(Nat[:Nfft//2]))
plt.xscale('log');plt.yscale('log')

Z.taper(max_percentage=0.05)
Nhat = fft(N[0].data,n=Nfft)#/np.sqrt(Z[0].stats.npts)

plt.plot(freqVec,np.abs(Nhat[:Nfft//2]))
plt.xscale('log');plt.yscale('log')

**f. Short Time Fourier Transform (3 points)**

STFT are important transforms that are used in data science of time series. They are mainly used for denoising and for feature extraction.
Spectrograms are STFT with window overlap.

In [ ]:
# answer below
from scipy.signal import stft
f, t, Zxx = stft(n, fs=100,noverlap=200)


Now you have created a 2D image of a time series! Many seismologists use that as input to convolutional neural networks.



## 2. Time Series analysis of Hunga- Tonga
 Ghent and Crowell, 2022: TF representation of the TEC (https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2022GL100145). The Total Electron Content (TEC) is the total number of electrons present along a path between a radio transmitter and receiver. TEC is measured in electrons per square meter. By convention, 1 TEC Unit TECU = 10^16 electrons/m². Vertical TEC values in Earth’s ionosphere can range from a few to several hundred TECU.

 On 15 January 2022, Tonga's Hunga Tonga-Hunga Ha'apai (HTHH) volcano violently erupted, generating a tsunami that killed three people. Acoustic-gravity waves propagated by the eruption and tsunami caused global complex ionospheric disturbances. In this paper, we study the nature of these perturbations from Global Navigation Satellite System observables over the southwestern Pacific. After processing data from 818 ground stations, we detect supersonic acoustic waves, Lamb waves, and tsunamis, with filtered magnitudes between 1 and 7 Total Electron Content units.

Apply the Fourier and Wavelet transforms to the TEC time series observed a a GPS section..


### a. Read the data (1 point)

Read the CSV file file ``./TEC/ioncorr_samo_015_2022_tonga.csv``, print the keys of the data frame. THe key ``time`` is a timestamp of seconds since epoch time (1970/01/01). The data shown in the article is in the Series ``variometric_derived_TEC``. **samo** is the name of the GPS receiver. There are up to 32 satellite data saved in the CSV file.

Plot the first 2 hours of the time series

In [ ]:
# answer below
tonga_path = '/Users/cristianswift/Desktop/Fall-Quarter-2022-2023/Machine_Learning_for_Geoscientist/NLGeo2022_cjswift/homework/homework 4/ioncorr_samo_015_2022_tonga.csv'
tonga = pd.read_csv(tonga_path)
print(tonga.keys())

In [ ]:
tonga.head()

## b. Get station info (0.5 point)
Get the station name by splitting the string of the file name and getting the charcater after "ioncorr". You can use the python function ``split()``.

In [ ]:
# answer below
tonga_path.split('ioncorr_')[1]

## c. Plot all data (2 points)
Make a plot of all vTEC filtered (``variometric_derived_TEC_filtered``) for each satellite. Align the data with the satellite number.

Use the ``plot_date`` function from matplotlib. Convert the Series ``time`` from a ``timestamp`` to an Numpy array of dtpe ``datetime64[s]`` (time stamps are in seconds, so we need to use ``[s]``). Plot the data for each sattelite observations.

In [ ]:
tonga.time = tonga.time.values.astype('datetime64[s]')

In [ ]:
unqiue_satellite = tonga['satellite_number'].unique()
len(unqiue_satellite)
    

In [ ]:
unqiue_satellite = tonga['satellite_number'].unique()

i,j=0,0
fig, axs = plt.subplots(5,5, figsize=(20, 60))

plots_per_row = 5

for sat in unqiue_satellite:
    temp_df = tonga[tonga['satellite_number'] == sat]
    
    axs[i][j].plot_date(temp_df['time'], temp_df['variometric_derived_TEC'])
    axs[i][j].set_xlabel(str('Satellite' + str(sat)))
    
    j+=1
    if j%plots_per_row==0:
        i+=1
        j=0
    

plt.show()

You can now select one of the satellite observations that contain a full vTEC signal.

In [ ]:
# I Choose Satellite23

### d. Perform the wavelet transform (3 points)

Just like in the article, perform the wavelet transform using a Morlet transform. Select the time series of interest and plot the time series. What can you tell about the time-frequency characteristic of these disturbances?

In [ ]:
# answer below
sat23 = tonga[tonga['satellite_number'] == 23]
fig, axs = plt.subplots(1, figsize=(6, 8))
axs.plot_date(sat23['time'], sat23['variometric_derived_TEC'])

Calculate the sampling frequency by taking the time difference between two samples.

In [ ]:
#answer below
freq = sat23['time'].iloc[1] - sat23['time'].iloc[0]
print('The frequency is', freq, 'seconds')

Perform the wavelet transform. You may choose a range of wavelet scales from 1 to 50 (write this as an array of integer), and call the ``cwt`` functions using the Morlet wavelet and taking the array of scales as an input argument. Plot it with an x-axis in time (hours) and y-axis in periods.

In [ ]:
import scipy.signal as signal
# use the number of scales
w = range(1,50)
widths = w*fs / (2*freq*np.pi)
# answer below


## e. Interpretation (0.5 points)
Can you describe the spectral features as a function of hours since the beginning of the time series? What periods dominate when?

(answer below)

The authors interpret the first packet as coming from a Lamb wave (a powerful gravity-acoustic wave that travel in the lower atmosphere), the second as coming from the tsunami disturbance.

## 3) 2D Spectral analysis of geological models (10 points)

In this exercise we will correlate water table level with surface elevation. You may download the data just like in the class. The file names are ``NCM_GeologicFrameworksGrids.nc``, ``NCM_SpatialGrid.nc``, and ``NCM_AuxData.nc``.

We first download the data from our Dropbox folder


In [ ]:
import wget
file1 = wget.download("https://www.dropbox.com/s/wdb25puxh3u07dj/NCM_GeologicFrameworkGrids.nc?dl=1") #"./data/NCM_GeologicFrameworkGrids.nc"
# Download the coordinate grids
file2 = wget.download("https://www.dropbox.com/s/i6tv3ug15oe6yhe/NCM_SpatialGrid.nc?dl=1") #"./data/NCM_GeologicFrameworkGrids.nc"
# Download the coordinate grids
file3 = wget.download("https://www.dropbox.com/s/92m20pehfu7rxp2/NCM_AuxData.nc?dl=1") #"./data/NCM_AuxData.nc"


In [ ]:
#os.makedirs('../../data_hw4_3/')
os.replace(file1,'../../data/'+file1)
os.replace(file2,'../../data/'+file2)
os.replace(file3,'../../data/'+file3)


In the following we will prepare our data. Read the DataSets using the python NetCDF4 library.

In [ ]:
# ansert below
import netCDF4 as nc

# read data
geology = nc.Dataset('../../data/'+file1)
grid = nc.Dataset('../../data/'+file2)
aux = nc.Dataset('../../data/'+file3)

In [ ]:
grid.variables.keys()

**a. Plot (2 points)**

Plot the data ``WT`` and ``elevation``, which are data sets from the NC files. Use the matplotlib function``contourf``, and the variables for lat long ``x`` and ``y`` . You can use the argument ``levels`` to the contourf funxtion to split the color map into discrete contour levels, and the transparency argument ``alpha`` to be less than 1.

In [ ]:
# answer below
# create a grid of latitude and longitude
x = grid['x'][0:4901, 0:3201]
y = grid['y'][0:4901, 0:3201]
WT = aux['Water Table Depth'][0:4901, 0:3201]
plt.contourf(x, y, WT, list(range(0,5)), alpha=0.9)



In [ ]:
# answer below
# create a grid of latitude and longitude
x = grid['x'][0:4901, 0:3201]
y = grid['y'][0:4901, 0:3201]
elevation = geology['Surface Elevation'][0:4901, 0:3201]
plt.contourf(x, y, elevation, list(range(0,5)))

elevation = geology['Surface Elevation'][0:4901, 0:3201]


**b. Perform and plot the 2D Fourier transforms (4 points)**

In [ ]:
#First for elevation

In [ ]:
# answer below
from scipy.fftpack import fft2, fftfreq,fftshift, ifft2
import matplotlib.cm as cm

Zel = fft2(elevation)



# make a vector of distances. Here I will ignore the curvature and spatial projection.
# make the wavenumber frequency vector: 
Rlon = (xlon-np.min(xlon))*111.25  # convert degrees to kms
drlon = Rlon[1]-Rlon[0]
print("this is about the spatial sampling of the model ",drlon," km")
klon = (fftfreq( 4901//2 , drlon  ))


Rlat = (xlat-np.min(xlat))*111.25  # convert degrees to kms
drlat = Rlat[1]-Rlat[0]
print("this is about the spatial sampling of the model ",drlat," km")
klat = (fftfreq( 3201//2 , drlat  ))

# amplitude of the DEM
plt.imshow(fftshift(np.log10(np.abs(Zel)/Zel.size)),vmin=-3, vmax=-1, cmap='RdYlBu',extent=[-1,1,-1,1])
plt.title('2D FT of elevation')
x_label_list = ['-1/3 km$^{-1}$','0','1/3  km$^{-1}$']
plt.xticks([-0.5,0,0.5])
plt.yticks([-0.5,0,0.5])
plt.show()

In [ ]:
# Now for WT 

In [ ]:

# answer below
from scipy.fftpack import fft2, fftfreq,fftshift, ifft2
import matplotlib.cm as cm

Zel = fft2(WT)



# make a vector of distances. Here I will ignore the curvature and spatial projection.
# make the wavenumber frequency vector: 
Rlon = (xlon-np.min(xlon))*111.25  # convert degrees to kms
drlon = Rlon[1]-Rlon[0]
print("this is about the spatial sampling of the model ",drlon," m")
klon = (fftfreq( 4901//2 , drlon  ))


Rlat = (xlat-np.min(xlat))*111.25  # convert degrees to kms
drlat = Rlat[1]-Rlat[0]
print("this is about the spatial sampling of the model ",drlat," m")
klat = (fftfreq( 3201//2 , drlat  ))

# amplitude of the DEM
plt.imshow(fftshift(np.log10(np.abs(Zel)/Zel.size)),vmin=-3, vmax=-1, cmap='RdYlBu',extent=[-1,1,-1,1])
plt.title('2D FT of Water Table Depth')
x_label_list = ['-1/3 km$^{-1}$','0','1/3  m$^{-1}$']
plt.xticks([-0.5,0,0.5])
plt.yticks([-0.5,0,0.5])
plt.show()

**c. Interpretation (1 point)**

Comment on the wavelengths that dominate the DEM and the water table wavelengths

$####### CRYTVUBINOMO;LNJBHGCVLKJ HLJN ANSWERRRR

**d. 2D filtering (3 points)**

Find a way to low pass filter the image (spectral filtering or convolution)

In [ ]:
from obspy import read
import obspy.signal
from obspy.signal import filter
# sampling rate of the data is 0.01 and below being a lowpass filter:
tr = Z[0] 
tr.data = obspy.signal.filter.lowpass(
    tr.data, freq=0.01, df=tr.stats.sampling_rate)

z=np.asarray(tr.data)
n=np.asarray(tr.data)

Now we will filter or compress by taking the largest Fourier coefficients of the image.

In [ ]:
# Sort the Fourier coefficients
Zel = fft2(Z)
Zsort = np.sort(np.abs(np.abs(Zel).reshape(-1)))

Plot and reconstruct the image of the water table map with 1% until 10% of the data (like in class)

In [ ]:
from IPython import display
import time
for keep in (0.1,0.05,0.01):
    display.clear_output(wait=True)
    thresh = Zsort[int(np.floor( (1-keep)*len(Zsort) ))]
    ind = np.abs(Zel)>thresh
    Atlow = tr[0] * ind # here we zero out the matrix
    # Here we count the number of non-zeros in the matrix
    print("We are keeping up to %f the number of Fourier coefficients" % keep)
    Alow = ifft2(Atlow).real
    plt.contourf(x, y, Alow)
    time.sleep(1)

Now we will compare the original 2D data set with the Fourier compressed data


In [ ]:
keep=0.01
thresh = Zsort[int(np.floor( (1-keep)*len(Zsort) ))]
ind = np.abs(Zel)>thresh
Atlow = Zel * ind # here we zero out the matrix
# Here we count the number of non-zeros in the matrix
print("We are keeping up to %f the number of Fourier coefficients" % keep)
Alow = ifft2(Atlow).real


fig,ax=plt.subplots(1,2,figsize=(8,8),sharex=True)
ax[0].contourf(x, y, elevation);ax[0].set_title('Original data')
ax[0].axis('scaled')
ax[1].contourf(x, y, Alow);ax[1].set_title('Compressed data')
ax[1].axis('scaled')